In [43]:
import pandas as pd
import numpy as np
import wandb

In [44]:
df = pd.read_csv("..//src//data_ingestion//stock_data.csv" )

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1006 non-null   object 
 1   Open       1006 non-null   float64
 2   High       1006 non-null   float64
 3   Low        1006 non-null   float64
 4   Close      1006 non-null   float64
 5   Adj Close  1006 non-null   float64
 6   Volume     1006 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 55.1+ KB


In [46]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [47]:
df.head()

,date,open,high,low,close,adj_close,volume
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.059433,135480400
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.349129,146322800
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,72.925636,118387200
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.582657,108872000
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,73.750244,132079200


In [51]:
df["tomorrow"] = df["close"].shift(-1)
df["label"] = (df["tomorrow"] > df["close"]).astype(int)
df.head()

,date,open,high,low,close,adj_close,volume,tomorrow,label
0,2020-01-02,74.059998,75.150002,73.797501,75.087502,73.059433,135480400,74.357498,0
1,2020-01-03,74.287498,75.144997,74.125000,74.357498,72.349129,146322800,74.949997,1
2,2020-01-06,73.447502,74.989998,73.187500,74.949997,72.925636,118387200,74.597504,0
3,2020-01-07,74.959999,75.224998,74.370003,74.597504,72.582657,108872000,75.797501,1
4,2020-01-08,74.290001,76.110001,74.290001,75.797501,73.750244,132079200,77.407501,1


In [27]:
for i in range(1 , 7 , 1) :
    df[f"lag_{i}_close"] = df["close"].shift(i)

In [29]:
df.tail()

,date,open,high,low,close,adj_close,volume,lag_1_close,lag_2_close,lag_3_close,lag_4_close,lag_5_close,lag_6_close
1001,2023-12-22,195.179993,195.410004,192.970001,193.600006,193.353287,37122800,194.679993,194.830002,196.940002,195.889999,197.570007,198.110001
1002,2023-12-26,193.610001,193.889999,192.830002,193.050003,192.803986,28919300,193.600006,194.679993,194.830002,196.940002,195.889999,197.570007
1003,2023-12-27,192.490005,193.500000,191.089996,193.149994,192.903839,48087700,193.050003,193.600006,194.679993,194.830002,196.940002,195.889999
1004,2023-12-28,194.139999,194.660004,193.169998,193.580002,193.333298,34049900,193.149994,193.050003,193.600006,194.679993,194.830002,196.940002
1005,2023-12-29,193.899994,194.399994,191.729996,192.529999,192.284637,42628800,193.580002,193.149994,193.050003,193.600006,194.679993,194.830002


In [33]:
import pandas_ta as ta

df['rsi_14'] = ta.rsi(df['close'], length=14)

df.tail(50)

,date,open,high,low,close,adj_close,volume,lag_1_close,lag_2_close,lag_3_close,lag_4_close,lag_5_close,lag_6_close,rsi_14
956,2023-10-19,176.039993,177.839996,175.190002,175.460007,175.005844,59302900,175.839996,177.149994,178.720001,178.850006,180.710007,179.800003,45.752238
957,2023-10-20,175.309998,175.419998,172.639999,172.880005,172.432510,64189300,175.460007,175.839996,177.149994,178.720001,178.850006,180.710007,40.297970
958,2023-10-23,170.910004,174.009995,169.929993,173.000000,172.552200,55980100,172.880005,175.460007,175.839996,177.149994,178.720001,178.850006,40.652339
959,2023-10-24,173.050003,173.669998,171.449997,173.440002,172.991058,43816600,173.000000,172.880005,175.460007,175.839996,177.149994,178.720001,42.011543
960,2023-10-25,171.880005,173.059998,170.649994,171.100006,170.657120,57157000,173.440002,173.000000,172.880005,175.460007,175.839996,177.149994,37.139989
961,2023-10-26,170.369995,171.380005,165.669998,166.889999,166.458008,70625300,171.100006,173.440002,173.000000,172.880005,175.460007,175.839996,30.326449
962,2023-10-27,166.910004,168.960007,166.830002,168.220001,167.784561,58499100,166.889999,171.100006,173.440002,173.000000,172.880005,175.460007,34.419613
963,2023-10-30,169.020004,171.169998,168.869995,170.289993,169.849197,51131000,168.220001,166.889999,171.100006,173.440002,173.000000,172.880005,40.298283
964,2023-10-31,169.350006,170.899994,167.899994,170.770004,170.327972,44846000,170.289993,168.220001,166.889999,171.100006,173.440002,173.000000,41.605490
965,2023-11-01,171.000000,174.229996,170.119995,173.970001,173.519684,56934900,170.770004,170.289993,168.220001,166.889999,171.100006,173.440002,49.537912


In [35]:
macd = ta.macd(df['close'])


In [36]:
macd.head(50)

,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [37]:
df['adl'] = (((df['close'] - df['close'].shift()) / (df['high'] - df['low'])) * df['volume']).fillna(0).cumsum()


In [40]:
df.tail()

,date,open,high,low,close,adj_close,volume,lag_1_close,lag_2_close,lag_3_close,lag_4_close,lag_5_close,lag_6_close,rsi_14,adl
1001,2023-12-22,195.179993,195.410004,192.970001,193.600006,193.353287,37122800,194.679993,194.830002,196.940002,195.889999,197.570007,198.110001,54.609714,6.305361e+09
1002,2023-12-26,193.610001,193.889999,192.830002,193.050003,192.803986,28919300,193.600006,194.679993,194.830002,196.940002,195.889999,197.570007,53.026883,6.290355e+09
1003,2023-12-27,192.490005,193.500000,191.089996,193.149994,192.903839,48087700,193.050003,193.600006,194.679993,194.830002,196.940002,195.889999,53.291938,6.292351e+09
1004,2023-12-28,194.139999,194.660004,193.169998,193.580002,193.333298,34049900,193.149994,193.050003,193.600006,194.679993,194.830002,196.940002,54.481470,6.302177e+09
1005,2023-12-29,193.899994,194.399994,191.729996,192.529999,192.284637,42628800,193.580002,193.149994,193.050003,193.600006,194.679993,194.830002,51.061838,6.285413e+09


In [41]:
# Create DataFrame with example data
data = pd.DataFrame({
    'Date': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05'],
    'Close': [100, 105, 102, 110, 108],
    'Volume': [200000, 150000, 180000, 120000, 160000]
})

# Calculate OBV
data['OBV'] = (data['Close'].diff() > 0).astype(int) * data['Volume'].diff().fillna(0).cumsum()

# Display the data with OBV
print(data)

         Date  Close  Volume      OBV
0  2022-01-01    100  200000      0.0
1  2022-01-02    105  150000 -50000.0
2  2022-01-03    102  180000     -0.0
3  2022-01-04    110  120000 -80000.0
4  2022-01-05    108  160000     -0.0
